In [1]:
import pandas as pd
import numpy as np
import re
import operator
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

# Read in the data

The data is stored in three different files so we need to read them in and merge them.

In [2]:
df_1 = pd.read_csv('avalik_1.csv', sep='\t')
df_2 = pd.read_csv('avalik_2.csv', sep='\t')
df_3 = pd.read_csv('avalik_3.csv', sep='\t')

In [3]:
df = df_1.append(df_2).append(df_3)
df = df.drop_duplicates()

In [4]:
df.head()

,JuhtumId,ToimKpv,ToimKell,ToimNadalapaev,SyndmusLiik,SyndmusTaiendavStatLiik,Seadus,Paragrahv,ParagrahvTais,Loige,Kahjusumma,KohtLiik,MaakondNimetus,ValdLinnNimetus,KohtNimetus,Lest_X,Lest_Y,SyyteoLiik
0,cf668bac-6136-18d8-83cf-eb3a78c60192,2020-09-24,02:11,Neljapäev,PISIVARGUS,NaN,Karistusseadustik,§ 218.,§ 218. Varavastane süütegu väheväärtusliku asj...,lg. 1.,0-499,"AVALIK_KOHT,KAUPLUS",Harju maakond,Tallinn,Lasnamäe linnaosa,6587500-6587999,544500-544999,VT
1,cf668b84-6136-18d8-83cf-eb3a78c60192,2020-09-23,20:03,Kolmapäev,PISIVARGUS,NaN,Karistusseadustik,§ 218.,§ 218. Varavastane süütegu väheväärtusliku asj...,lg. 1.,0-499,"AVALIK_KOHT,KAUPLUS",Harju maakond,Tallinn,Haabersti linnaosa,6587500-6587999,537000-537499,VT
2,961bedb4-606a-18d8-83cf-eb3a78c60192,2020-09-23,19:08,Kolmapäev,VARGUS,MUU_VARGUS,Karistusseadustik,§ 199.,§ 199. Vargus,lg. 2.,0-499,"AVALIK_KOHT,KAUPLUS",Harju maakond,Tallinn,Mustamäe linnaosa,6585500-6585999,539000-539499,KT
3,961bed6e-606a-18d8-83cf-eb3a78c60192,2020-09-23,18:32,Kolmapäev,VARGUS,MOBIILTELEFONIVARGUS,Karistusseadustik,§ 199.,§ 199. Vargus,lg. 2.,500-4999,"AVALIK_KOHT,SOOGIKOHT",Harju maakond,Tallinn,Põhja-Tallinna linnaosa,6589000-6589499,541000-541499,KT
4,961bed46-606a-18d8-83cf-eb3a78c60192,2020-09-23,17:55,Kolmapäev,JALGRATTA_MOPEEDI_VARGUS,JALGRATTAVARGUS,Karistusseadustik,§ 199.,§ 199. Vargus,lg. 1.,0-499,"AVALIK_KOHT,TANAV_VALJAK",Harju maakond,Tallinn,Kesklinna linnaosa,6588500-6588999,542500-542999,KT


# Cleaning the data

Let's now prepare the data so that it will be useable for the regression model. It is also in Estonian so I will translate it as well (at least most of it).

In [5]:
df['ToimKpv'] = pd.to_datetime(df['ToimKpv'])

In [6]:
# only keep events in Tallinn
df = df.loc[df['ValdLinnNimetus'].str.contains('Tallinn', regex=True, na=False)]
# make sure 'linnaosa' (district) is also filled in
df = df.loc[df['KohtNimetus'].str.contains('linnaosa', regex=True, na=False)]

In [7]:
# keep only events connected to mobile phones
df['is_mobile'] = np.where(df['SyndmusTaiendavStatLiik'].str.contains('MOBIIL', na=False), 1, 0)

In [8]:
# keep only necessary columns and translate them
df = df[['ToimKpv', 'ToimKell', 'ToimNadalapaev', 'KohtLiik', 'KohtNimetus', 'is_mobile']]
df = df.rename(columns={'ToimKpv':'date', 'ToimKell':'time', 'ToimNadalapaev':'weekday', 
                        'KohtLiik':'place', 'KohtNimetus':'district'})

In [9]:
# remove whitespace and empty values
for col in ['weekday', 'place', 'district']:
    df[col] = df[col].str.strip()
df = df.dropna()

In [10]:
# for 'place', remove too generic values like 'public place', 'street' and 'other'
for value in ['AVALIK_KOHT', 'TANAV_VALJAK', 'MUU KOHT', 'MUU RUUM']:
    df['place'] = df['place'].str.replace(value+',','')
    df['place'] = df['place'].str.replace(','+value,'')

In [11]:
# if there are multiple places separated by a comma, keep only the first one
df['place'] = np.where(df['place'].str.contains(','),
                       df['place'].str.extract('^(\w+),')[0],
                       df['place'])

In [12]:
# translate weekdays
df['weekday'] = df['weekday'].map({'Esmaspäev':'Mon', 'Teisipäev':'Tue', 
                                   'Kolmapäev':'Wed', 'Neljapäev':'Thu', 
                                   'Reede':'Fri', 'Laupäev':'Sat', 'Pühapäev':'Sun'})

In [13]:
# extract season to be used by the model instead the specific month
df['month'] = pd.to_datetime(df['date']).dt.month_name()
df['season'] = df['month'].map({'December':'winter','January':'winter', 'February':'winter',
                                'March':'spring', 'April':'spring', 'May':'spring',
                                'June':'summer', 'July':'summer', 'August':'summer',
                                'September':'autumn', 'October':'autumn', 'November':'autumn'})

In [14]:
# extract time of day
df['hour'] = pd.to_datetime(df['time'], format='%H:%M').dt.hour
df['timeofday'] = np.where((df['hour']>=6)&(df['hour']<=17),'day','night')
df = df.drop(columns=['hour', 'time', 'month'])

In [15]:
df = df.reset_index(drop=True)

In [16]:
print(len(df)) # 63896
df.head()

63896


,date,weekday,place,district,is_mobile,season,timeofday
0,2020-09-24,Thu,KAUPLUS,Lasnamäe linnaosa,0,autumn,night
1,2020-09-23,Wed,KAUPLUS,Haabersti linnaosa,0,autumn,night
2,2020-09-23,Wed,KAUPLUS,Mustamäe linnaosa,0,autumn,night
3,2020-09-23,Wed,SOOGIKOHT,Põhja-Tallinna linnaosa,1,autumn,night
4,2020-09-23,Wed,TANAV_VALJAK,Kesklinna linnaosa,0,autumn,day


# Data set description

Most of the cases are not connected to mobile phones so we need to keep that in mind when we build the model.

In [17]:
df['is_mobile'].value_counts()

0    59129
1     4767
Name: is_mobile, dtype: int64

Also the number of cases registered overall is decreasing year by year.

In [18]:
df['date'].dt.year.value_counts()

2012    10306
2013     9949
2014     8521
2015     8310
2016     7081
2017     5893
2018     5583
2019     4956
2020     3297
Name: date, dtype: int64

# Regression 2019-2020

Let's now start with the regression analysis. I will first write a function that furhter prepares the data for the analysis.

The function will:
1. filter out data from the interested time period (2019-2020 or 2012-2013)
2. equalize the number of cases connected to mobiles and not connected to mobiles (otherwise the model will be too biased towards cases not connected to mobiles)
3. filter out top 8 values for 'place' to keep the number of variables under control
4. get dummies for all the categorical variables (which is all of them actually)
5. calculate the benchmark dummies for each category

In [19]:
def data_for_model(df_in, years):
    df_out = df_in.loc[df_in['date'].dt.year.isin(years)]
    df_out = df_out.drop(columns=['date'])
    
    n_to_drop = df_out['is_mobile'].value_counts()[0] - df_out['is_mobile'].value_counts()[1]
    df_out = df_out.drop(df_out.query('is_mobile == 0').sample(n=n_to_drop, random_state=42).index)
    
    top_places = df_out['place'].value_counts().head(8).index
    df_out = df_out.loc[df_out['place'].isin(top_places)]
    
    categories = []
    for col in ['weekday', 'place', 'district', 'season', 'timeofday']:
        categories.append(list(df_out[col].unique()))
    categories = [item for sublist in categories for item in sublist]
    
    df_out = pd.get_dummies(df_out, drop_first=True)
    
    dummies = list(df_out.columns.values)
    dummies.remove('is_mobile')
    dummies = [i[i.find('_')+1:] for i in dummies]
    
    benchmarks = [cat for cat in categories if cat not in dummies]
    
    return df_out, benchmarks

In [20]:
df_19 = data_for_model(df, ['2019', '2020'])[0]
benchmarks_19 = data_for_model(df, ['2019', '2020'])[1]

Let's also translate place values.

In [21]:
[col for col in df_19.columns if 'place_' in col]

['place_KAUBAMAJA',
 'place_KAUPLUS',
 'place_OOKLUBI_DISKOTEEK',
 'place_SOOGIKOHT',
 'place_TANAV_VALJAK',
 'place_TANKLA',
 'place_UHISSOIDUK']

In [22]:
df_19 = df_19.rename(columns={'place_KAUBAMAJA':'place_shopping_centre', 'place_KAUPLUS':'place_shop', 
               'place_OOKLUBI_DISKOTEEK':'place_nightclub', 'place_SOOGIKOHT':'place_restaurant',
               'place_TANAV_VALJAK':'place_street_square', 'place_TANKLA':'place_gas_station', 
               'place_UHISSOIDUK':'place_public_transport'})

Let's split the data set to training and testing data sets, by 80% and 20% respectively.

In [23]:
df_train_19, df_test_19 = train_test_split(df_19, test_size=0.2, random_state=42)

And now let's train the model.

In [24]:
features = list(df_19.columns)
features.remove('is_mobile')

y = df_train_19['is_mobile']
x1 = df_train_19[features]

X = sm.add_constant(x1)
results_19 = sm.Logit(y,X).fit()

results_19.summary()

Optimization terminated successfully.
         Current function value: 0.373823
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              is_mobile   No. Observations:                  552
Model:                          Logit   Df Residuals:                      527
Method:                           MLE   Df Model:                           24
Date:                Sat, 17 Oct 2020   Pseudo R-squ.:                  0.4572
Time:                        16:38:17   Log-Likelihood:                -206.35
converged:                       True   LL-Null:                       -380.16
Covariance Type:            nonrobust   LLR p-value:                 3.816e-59
====================================================================================================
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                                2.5700      0.997      2.578      0.010       0.616       4.523
weekday_Mon                         -0.4091      0.501     -0.816      0.414      -1.391       0.573
weekday_Sat                          1.0183      0.414      2.460      0.014       0.207       1.830
weekday_Sun                          0.3733      0.433      0.861      0.389      -0.476       1.223
weekday_Thu                          0.0343      0.494      0.069      0.945      -0.935       1.003
weekday_Tue                         -0.5617      0.520     -1.081      0.280      -1.580       0.457
weekday_Wed                          0.0359      0.472      0.076      0.939      -0.890       0.962
place_shopping_centre               -2.0274      0.854     -2.375      0.018      -3.700      -0.354
place_shop                          -3.9580      0.723     -5.476      0.000      -5.375      -2.541
place_nightclub                      2.0656      1.237      1.670      0.095      -0.359       4.490
place_restaurant                     0.3590      0.743      0.483      0.629      -1.096       1.814
place_street_square                 -1.0502      0.692     -1.518      0.129      -2.406       0.306
place_gas_station                   -4.3307      1.244     -3.482      0.000      -6.769      -1.893
place_public_transport               0.9126      0.873      1.046      0.296      -0.798       2.623
district_Kesklinna linnaosa         -1.8030      0.633     -2.848      0.004      -3.044      -0.562
district_Kristiine linnaosa         -1.8373      0.844     -2.176      0.030      -3.492      -0.182
district_Lasnamäe linnaosa          -1.4971      0.639     -2.342      0.019      -2.750      -0.244
district_Mustamäe linnaosa          -3.0288      0.801     -3.779      0.000      -4.600      -1.458
district_Nõmme linnaosa             -2.4035      1.000     -2.403      0.016      -4.364      -0.443
district_Pirita linnaosa            -2.9611      1.220     -2.428      0.015      -5.351      -0.571
district_Põhja-Tallinna linnaosa    -1.8749      0.664     -2.825      0.005      -3.176      -0.574
season_spring                       -0.0853      0.407     -0.209      0.834      -0.884       0.713
season_summer                        0.8506      0.370      2.296      0.022       0.125       1.577
season_winter                        0.0086      0.418      0.021      0.984      -0.810       0.828
timeofday_night                      0.1626      0.274      0.594      0.552      -0.374       0.699
====================================================================================================
"""

### Removing insignificant features

As we can see, a number of the features are insignificant (p-value > 0.05). Let's remove them one by one, starting with the highest.

In [25]:
def remove_most_insignificant(df, results):
    # use operator to find the key which belongs to the maximum value in the dictionary:
    max_p_value = max(results.pvalues.iteritems(), key=operator.itemgetter(1))[0]
    # this is the feature you want to drop:
    df.drop(columns = max_p_value, inplace = True)
    return df

In [26]:
insignificant_feature = True

while insignificant_feature:
        results_19 = sm.Logit(y,X).fit()
        significant = [p_value < 0.05 for p_value in results_19.pvalues]
        if all(significant):
            insignificant_feature = False
        else:
            if X.shape[1] == 1:  # if there's only one insignificant variable left
                print('No significant features found')
                results_19 = None
                insignificant_feature = False
            else:            
                X = remove_most_insignificant(X, results_19)

Optimization terminated successfully.
         Current function value: 0.373823
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.373824
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.373828
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.373831
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.373897
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.374095
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.374486
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.375211
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.376059
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.377462
  

And here is our final model with all insignificant features removed.

In [27]:
results_19.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              is_mobile   No. Observations:                  552
Model:                          Logit   Df Residuals:                      538
Method:                           MLE   Df Model:                           13
Date:                Sat, 17 Oct 2020   Pseudo R-squ.:                  0.4430
Time:                        16:38:17   Log-Likelihood:                -211.75
converged:                       True   LL-Null:                       -380.16
Covariance Type:            nonrobust   LLR p-value:                 4.579e-64
====================================================================================================
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                                3.2385      0.643      5.037      0.000       1.978       4.499
weekday_Sat                          1.0214      0.305      3.344      0.001       0.423       1.620
place_shopping_centre               -2.7439      0.573     -4.792      0.000      -3.866      -1.622
place_shop                          -4.6217      0.398    -11.610      0.000      -5.402      -3.841
place_street_square                 -1.7072      0.309     -5.524      0.000      -2.313      -1.102
place_gas_station                   -4.9159      1.075     -4.572      0.000      -7.024      -2.808
district_Kesklinna linnaosa         -1.7410      0.623     -2.792      0.005      -2.963      -0.519
district_Kristiine linnaosa         -1.8290      0.837     -2.185      0.029      -3.469      -0.189
district_Lasnamäe linnaosa          -1.5161      0.634     -2.390      0.017      -2.760      -0.273
district_Mustamäe linnaosa          -3.0353      0.792     -3.835      0.000      -4.587      -1.484
district_Nõmme linnaosa             -2.3377      0.995     -2.348      0.019      -4.289      -0.387
district_Pirita linnaosa            -3.0486      1.219     -2.501      0.012      -5.437      -0.660
district_Põhja-Tallinna linnaosa    -1.8174      0.653     -2.784      0.005      -3.097      -0.538
season_summer                        0.8669      0.262      3.311      0.001       0.354       1.380
====================================================================================================
"""

Let's also compose a confusion matrix for the model's results.

In [28]:
def df_confusion_matrix(cm_array):
    df_cm = pd.DataFrame(cm_array)
    df_cm.columns = ['Predicted 0', 'Predicted 1']
    df_cm = df_cm.rename(index={0:'Actual 0', 1:'Actual 1'})
    
    print(f'model accuracy: {((cm_array[0,0] + cm_array[1,1])/cm_array.sum()).round(4)}')
    print(f'ppv: {(cm_array[1,1]/cm_array[1,:].sum()).round(4)}')
    print(f'npv: {(cm_array[0,0]/cm_array[0,:].sum()).round(4)}')
    print(f'sensitivity: {(cm_array[1,1]/cm_array[:,1].sum()).round(4)}')
    print(f'specificity: {(cm_array[0,0]/cm_array[:,0].sum()).round(4)}')
    
    return df_cm

In [29]:
def test_results(data, actual_values, model):
        pred_values = model.predict(data)
        bins=np.array([0,0.5,1])
        cm = np.histogram2d(actual_values, pred_values, bins=bins)[0]
        return cm

In [30]:
df_confusion_matrix(results_19.pred_table())

model accuracy: 0.8261
ppv: 0.812
npv: 0.8377
sensitivity: 0.8056
specificity: 0.8433


,Predicted 0,Predicted 1
Actual 0,253.0,49.0
Actual 1,47.0,203.0


The model seems to be quite good. All the metrics are above 80% which is nice.

### Testing the model

In [31]:
new_features = list(results_19.params.index)
new_features.remove('const')

test_actual_19 = df_test_19['is_mobile']
test_data_19 = df_test_19[new_features]
test_data_19 = sm.add_constant(test_data_19)

In [32]:
cm_test = test_results(test_data_19, test_actual_19, results_19)
df_confusion_matrix(cm_test)

model accuracy: 0.7986
ppv: 0.7333
npv: 0.8481
sensitivity: 0.7857
specificity: 0.8072


,Predicted 0,Predicted 1
Actual 0,67.0,12.0
Actual 1,16.0,44.0


Testing results are lower but sill not bad.

# Regression 2012-2013

Let's now repeat the same process but for data from 2012-2013.

In [33]:
df_12 = data_for_model(df, ['2012', '2013'])[0]
benchmarks_12 = data_for_model(df, ['2012', '2013'])[1]

In [34]:
[col for col in df_12.columns if 'place_' in col]

['place_OOKLUBI_DISKOTEEK',
 'place_PARKLA',
 'place_RIIDEHOID',
 'place_SOOGIKOHT',
 'place_TANAV_VALJAK',
 'place_TANKLA',
 'place_UHISSOIDUK']

In [35]:
df_12 = df_12.rename(columns={'place_OOKLUBI_DISKOTEEK':'place_nightclub',
                              'place_PARKLA':'place_car_park',
                              'place_RIIDEHOID':'place_cloakroom',
                              'place_SOOGIKOHT':'place_restaurant',
                              'place_TANAV_VALJAK':'place_street_square',
                              'place_TANKLA':'place_gas_station', 
                              'place_UHISSOIDUK':'place_public_transport'})

In [36]:
df_train_12, df_test_12 = train_test_split(df_12, test_size=0.2, random_state=42)

In [37]:
features = list(df_12.columns)
features.remove('is_mobile')

y = df_train_12['is_mobile']
x1 = df_train_12[features]

X = sm.add_constant(x1)
results = sm.Logit(y,X).fit()

Optimization terminated successfully.
         Current function value: 0.446665
         Iterations 8


### Removing insignificant features

In [38]:
insignificant_feature = True

while insignificant_feature:
        results_12 = sm.Logit(y,X).fit()
        significant = [p_value < 0.05 for p_value in results_12.pvalues]
        if all(significant):
            insignificant_feature = False
        else:
            if X.shape[1] == 1:  # if there's only one insignificant variable left
                print('No significant features found')
                results_12 = None
                insignificant_feature = False
            else:            
                X = remove_most_insignificant(X, results_12)

Optimization terminated successfully.
         Current function value: 0.446665
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.446677
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.446714
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.446752
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.446800
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.446861
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.447018
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.447216
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.447483
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.447661
  

In [39]:
results_12.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              is_mobile   No. Observations:                 2257
Model:                          Logit   Df Residuals:                     2243
Method:                           MLE   Df Model:                           13
Date:                Sat, 17 Oct 2020   Pseudo R-squ.:                  0.3509
Time:                        16:38:18   Log-Likelihood:                -1012.3
converged:                       True   LL-Null:                       -1559.6
Covariance Type:            nonrobust   LLR p-value:                7.802e-226
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                          -1.8590      0.141    -13.220      0.000      -2.135      -1.583
weekday_Mon                    -0.3858      0.177     -2.183      0.029      -0.732      -0.039
weekday_Thu                    -0.5366      0.172     -3.121      0.002      -0.874      -0.200
weekday_Tue                    -0.5186      0.174     -2.987      0.003      -0.859      -0.178
weekday_Wed                    -0.4148      0.185     -2.245      0.025      -0.777      -0.053
place_nightclub                 4.1660      0.315     13.213      0.000       3.548       4.784
place_car_park                  1.3397      0.226      5.932      0.000       0.897       1.782
place_cloakroom                 3.6626      0.371      9.881      0.000       2.936       4.389
place_restaurant                3.6081      0.216     16.685      0.000       3.184       4.032
place_street_square             1.7116      0.150     11.394      0.000       1.417       2.006
place_gas_station              -1.6435      0.728     -2.259      0.024      -3.070      -0.217
place_public_transport          3.9840      0.297     13.434      0.000       3.403       4.565
district_Kesklinna linnaosa     0.4292      0.124      3.465      0.001       0.186       0.672
district_Mustamäe linnaosa     -0.5688      0.275     -2.065      0.039      -1.109      -0.029
===============================================================================================
"""

In [40]:
df_confusion_matrix(results_12.pred_table())

model accuracy: 0.7926
ppv: 0.691
npv: 0.8819
sensitivity: 0.837
specificity: 0.7648


,Predicted 0,Predicted 1
Actual 0,1060.0,142.0
Actual 1,326.0,729.0


Here we can see a much lower PPV compared to the model for 2019-2020. This means the for those years, the model was able to correctly classify about 70% of the cases connected to mobile phones.

### Testing the model

In [41]:
new_features = list(results_12.params.index)
new_features.remove('const')

test_actual_12 = df_test_12['is_mobile']
test_data_12 = df_test_12[new_features]
test_data_12 = sm.add_constant(test_data_12)

In [42]:
cm_test = test_results(test_data_12, test_actual_12, results_12)
df_confusion_matrix(cm_test)

model accuracy: 0.7858
ppv: 0.7138
npv: 0.8547
sensitivity: 0.8243
specificity: 0.7577


,Predicted 0,Predicted 1
Actual 0,247.0,42.0
Actual 1,79.0,197.0


Testing results are very similar to training results.

# Summary

Let's now take a look at the summary of both models. What were the significant factors that ended up in both of the models? How have they changed over time?

In [43]:
summary_12 = pd.DataFrame(data=results_12.params.rename(2012))
summary_19 = pd.DataFrame(data=results_19.params.rename(2019))

summary = summary_12.merge(summary_19, right_index=True, left_index=True, how='outer')
summary

,2012,2019
const,-1.859018,3.238490
district_Kesklinna linnaosa,0.429159,-1.741017
district_Kristiine linnaosa,NaN,-1.829016
district_Lasnamäe linnaosa,NaN,-1.516064
district_Mustamäe linnaosa,-0.568813,-3.035289
district_Nõmme linnaosa,NaN,-2.337676
district_Pirita linnaosa,NaN,-3.048576
district_Põhja-Tallinna linnaosa,NaN,-1.817427
place_car_park,1.339720,NaN
place_cloakroom,3.662574,NaN


Here are also the benchmarka for the dummy variables. Kauplus means shop and bussipeatus is bus stop.

In [44]:
print(benchmarks_12)
print(benchmarks_19)

['Fri', 'KAUPLUS', 'Haabersti linnaosa', 'autumn', 'day']
['Fri', 'BUSSIPEATUS', 'Haabersti linnaosa', 'autumn', 'day']


## Conclusion

* It used to be that being in the city centre increased the odds of having your mobile phone stolen. These days, it is not so.
* There also used to be a number of places which increased the odds of having your phone stolen: car park, cloakroom, nightclub, public transport, restaurant, street. Today, none of those place have any meaningful impact on the odds any more.
* During 2019-2020, summertime and Saturday are increasing the odds. There used to be no difference in the season, and Mon-Thu decreased the odds (the baseline was Friday).
* Time of day (day/night) plays no significant role in neither of the time periods.